In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 55.0MB 58kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 5.5MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588515 sha256=7f73a4da2a2b25d13f76600e2b573002caf7261c1ece74a153ddc1bef6b3ee0c
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 4.0MB/s 
     |████████████████████████████████| 757kB 5.8MB/s 
     |████████████████████████████████| 3.0MB 30.9MB/s 
     |████████████████████████████████| 890kB 53.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=472cc94fa868a3fc416559676c859b285334ff165e1b9091be960b6c31e447cd
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a5012

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-c10fmhxw
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-c10fmhxw
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=d072dc78f0b461ff120e557af10d25d7b8441b293d379498b9d7693c2b7f5f3e
  Stored in directory: /tmp/pip-ephem-wheel-cache-tjmptsas/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
##GPU 사용 시
device = torch.device("cuda:0")

In [7]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [8]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2021-01-14 07:35:14--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2021-01-14 07:35:14--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc41cc2f72beaf7f91beec972098.dl.dropboxusercontent.com/cd/0/get/BG8orAHbIn4dhPDReV7tFVqAUEkzPjKl68E9r7zhE5ClMUoNrC-SOYQxx--auzN1cZklNFW8CyK2Vq4SB5B33ZTbR_NNvYfhEOU2gZ2FLS7LvYdXBpnb4Yks7XqxAuUNR-Y/file?dl=1# [following]
--2021-01-14 07:35:14--  https://uc41cc2f72beaf7f91beec972098.dl.dropboxusercontent.com/cd/0/get/BG8orAHbIn4dhPDReV7tFVqAUEkzPjKl68E9r7zhE5ClMUoNrC-SOYQxx--auzN1cZklNFW8CyK2Vq4SB5B33ZTbR_N

In [9]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [10]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [12]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [13]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [14]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [15]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [17]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [18]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [19]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [20]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [21]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [22]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 0.6934864521026611 train acc 0.59375
epoch 1 batch id 201 loss 0.46280717849731445 train acc 0.576181592039801
epoch 1 batch id 401 loss 0.49844810366630554 train acc 0.6776807980049875
epoch 1 batch id 601 loss 0.47399041056632996 train acc 0.7283953826955075
epoch 1 batch id 801 loss 0.4434967041015625 train acc 0.7572565543071161
epoch 1 batch id 1001 loss 0.3044055104255676 train acc 0.7749594155844156
epoch 1 batch id 1201 loss 0.2516319453716278 train acc 0.7889128850957535
epoch 1 batch id 1401 loss 0.3668358325958252 train acc 0.7982356352605282
epoch 1 batch id 1601 loss 0.31346672773361206 train acc 0.8065174109931293
epoch 1 batch id 1801 loss 0.2898726165294647 train acc 0.8133155191560244
epoch 1 batch id 2001 loss 0.3161255717277527 train acc 0.8194340329835083
epoch 1 batch id 2201 loss 0.3416569232940674 train acc 0.8244547932757837

epoch 1 train acc 0.8279161333902162


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.884550831202046


epoch 2 batch id 1 loss 0.47027960419654846 train acc 0.796875
epoch 2 batch id 201 loss 0.19433581829071045 train acc 0.8811411691542289
epoch 2 batch id 401 loss 0.27972549200057983 train acc 0.8824812967581047
epoch 2 batch id 601 loss 0.37610894441604614 train acc 0.8872972129783694
epoch 2 batch id 801 loss 0.3264349400997162 train acc 0.8895131086142322
epoch 2 batch id 1001 loss 0.33774781227111816 train acc 0.8912493756243757
epoch 2 batch id 1201 loss 0.19277597963809967 train acc 0.8934871981681932
epoch 2 batch id 1401 loss 0.16530844569206238 train acc 0.8951753211991434
epoch 2 batch id 1601 loss 0.29440101981163025 train acc 0.8971541224234854
epoch 2 batch id 1801 loss 0.1501418650150299 train acc 0.8989016518600778
epoch 2 batch id 2001 loss 0.2532915771007538 train acc 0.9008074087956022
epoch 2 batch id 2201 loss 0.22171884775161743 train acc 0.9024165152203544

epoch 2 train acc 0.9038302581058021


...


epoch 3 test acc 0.8979379795396419


epoch 4 batch id 1 loss 0.407063752412796 train acc 0.859375
epoch 4 batch id 201 loss 0.08295740187168121 train acc 0.9546019900497512
epoch 4 batch id 401 loss 0.06995470821857452 train acc 0.954839463840399
epoch 4 batch id 601 loss 0.17422311007976532 train acc 0.9567127703826955
epoch 4 batch id 801 loss 0.13705149292945862 train acc 0.9582553058676654
epoch 4 batch id 1001 loss 0.1491706669330597 train acc 0.959399975024975
epoch 4 batch id 1201 loss 0.07315324991941452 train acc 0.9608529350541216
epoch 4 batch id 1401 loss 0.032556772232055664 train acc 0.9620248929336188
epoch 4 batch id 1601 loss 0.18379086256027222 train acc 0.9627869300437226
epoch 4 batch id 1801 loss 0.061575308442115784 train acc 0.9635879372570794
epoch 4 batch id 2001 loss 0.09307236969470978 train acc 0.9644708895552224
epoch 4 batch id 2201 loss 0.21477097272872925 train acc 0.964994604725125

epoch 4 train acc 0.9656436646757679



epoch 4 test acc 0.8965593030690537


epoch 5 batch id 1 loss 0.37520840764045715 train acc 0.890625
epoch 5 batch id 201 loss 0.03998485580086708 train acc 0.9705379353233831
epoch 5 batch id 401 loss 0.03311261534690857 train acc 0.9735816708229427
epoch 5 batch id 601 loss 0.13046744465827942 train acc 0.9741576539101497
epoch 5 batch id 801 loss 0.08067353814840317 train acc 0.9750117041198502
epoch 5 batch id 1001 loss 0.02492067962884903 train acc 0.9760551948051948
epoch 5 batch id 1201 loss 0.01934047043323517 train acc 0.9765820149875104
epoch 5 batch id 1401 loss 0.013478746637701988 train acc 0.9767688258386866
epoch 5 batch id 1601 loss 0.09160381555557251 train acc 0.9770065584009994
epoch 5 batch id 1801 loss 0.067233607172966 train acc 0.9774257357023876
epoch 5 batch id 2001 loss 0.039951350539922714 train acc 0.9778548225887056
epoch 5 batch id 2201 loss 0.12264366447925568 train acc 0.9781136415265789

epoch 5 train acc 0.9783223122866894



epoch 5 test acc 0.8959998401534527
